In [1]:
#import relelvent packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


data = pd.read_pickle('arxiv_fulldataset.pkl')

### Author Analysis

In [2]:
#avg number of authors per paper
num_authors = [len(ls) for ls in data.authors_parsed]
data['num_authors'] = num_authors
data.num_authors.describe()

count    1.789907e+06
mean     4.166603e+00
std      2.042236e+01
min      1.000000e+00
25%      2.000000e+00
50%      3.000000e+00
75%      4.000000e+00
max      2.832000e+03
Name: num_authors, dtype: float64

About 4 authors per paper

In [2]:
import re
authors_new = [[re.sub('[^a-z]', '', (author[0] + author[1]).lower()) for author in ls] for ls in data.authors_parsed]
assert(len(authors_new) == len(data))

In [3]:
data['authors_cleaned'] = authors_new

In [6]:
data['authors_parsed'].head(20)

0     [[Balázs, C., ], [Berger, E. L., ], [Nadolsky,...
1              [[Streinu, Ileana, ], [Theran, Louis, ]]
2                                    [[Pan, Hongjun, ]]
3                                   [[Callan, David, ]]
4     [[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]
5                     [[Pong, Y. H., ], [Law, C. K., ]]
6     [[Corichi, Alejandro, ], [Vukasinac, Tatjana, ...
7                                [[Swift, Damian C., ]]
8     [[Harvey, Paul, ], [Merin, Bruno, ], [Huard, T...
9                             [[Ovchinnikov, Sergei, ]]
10      [[Cunningham, Clifton, ], [Dembele, Lassina, ]]
11                                   [[Choi, Dohoon, ]]
12               [[Choi, Dohoon, ], [Choie, YoungJu, ]]
13                                  [[Fujii, Koichi, ]]
14                               [[Stahn, Christian, ]]
15    [[Chang, Chao-Hsi, ], [Li, Tong, ], [Li, Xue-Q...
16    [[Mhlahlo, Nceba, ], [Buckley, David H., ], [D...
17                            [[Gustavsson, Andr

In [5]:
data['authors_cleaned'].head(20)

0                [balzsc, bergerel, nadolskypm, yuancp]
1                          [streinuileana, theranlouis]
2                                          [panhongjun]
3                                         [callandavid]
4                  [abushammalawael, torchinskyalberto]
5                                       [pongyh, lawck]
6     [corichialejandro, vukasinactatjana, zapatajosea]
7                                        [swiftdamianc]
8     [harveypaul, merinbruno, huardtracyl, rebulllu...
9                                   [ovchinnikovsergei]
10                  [cunninghamclifton, dembelelassina]
11                                         [choidohoon]
12                           [choidohoon, choieyoungju]
13                                        [fujiikoichi]
14                                     [stahnchristian]
15        [changchaohsi, litong, lixueqian, wangyuming]
16    [mhlahlonceba, buckleydavidh, dhillonvikrams, ...
17                                  [gustavssona

### Edit Abstracts
from analysis we know most articles are around 200 words

there are 176 categories

In [6]:
#get list of all categories used
categories = list(data.categories.str.split(' '))
num_categories = [len(ls) for ls in categories]
categories_ls = [category for ls in categories for category in ls]

In [7]:
assert(len(categories_ls) == sum(num_categories))

In [8]:
#get set of unique categories used
categories_set = set(categories_ls)
print(len(categories_set))

176


In [9]:
#theoretical categories set from the ArXiv taxonomy page
categories_theor = 'cs.AI, cs.AR, cs.CC, cs.CE, cs.CG, cs.CL, cs.CR, cs.CV, cs.CY, cs.DB, cs.DC, cs.DL, cs.DM, cs.DS, cs.ET, cs.FL, cs.GL, cs.GR, cs.GT, cs.HC, cs.IR, cs.IT, cs.LG, cs.LO, cs.MA, cs.MM, cs.MS, cs.NA, cs.NE, cs.NI, cs.OH, cs.OS, cs.PF, cs.PL, cs.RO, cs.SC, cs.SD, cs.SE, cs.SI, cs.SY, econ.EM, econ.GN, econ.TH, eess.AS, eess.IV, eess.SP, eess.SY, math.AC, math.AG, math.AP, math.AT, math.CA, math.CO, math.CT, math.CV, math.DG, math.DS, math.FA, math.GM, math.GN, math.GR, math.GT, math.HO, math.IT, math.KT, math.LO, math.MG, math.MP, math.NA, math.NT, math.OA, math.OC, math.PR, math.QA, math.RA, math.RT, math.SG, math.SP, math.ST, astro-ph.CO, astro-ph.EP, astro-ph.GA, astro-ph.HE, astro-ph.IM, astro-ph.SR, cond-mat.dis-nn, cond-mat.mes-hall, cond-mat.mtrl-sci, cond-mat.other, cond-mat.quant-gas, cond-mat.soft, cond-mat.stat-mech, cond-mat.str-el, cond-mat.supr-con, gr-qc, hep-ex, hep-lat, hep-ph, hep-th, math-ph, nlin.AO, nlin.CD, nlin.CG, nlin.PS, nlin.SI, nucl-ex, nucl-th, physics.acc-ph, physics.ao-ph, physics.app-ph, physics.atm-clus, physics.atom-ph, physics.bio-ph, physics.chem-ph, physics.class-ph, physics.comp-ph, physics.data-an, physics.ed-ph, physics.flu-dyn, physics.gen-ph, physics.hist-ph, physics.ins-det, physics.med-ph, physics.optics, physics.plasm-ph, physics.pop-ph, physics.soc-ph, physics.space-ph, quant-ph, q-bio.BM, q-bio.CB, q-bio.GN, q-bio.MN, q-bio.NC, q-bio.OT, q-bio.PE, q-bio.QM, q-bio.SC, q-bio.TO, q-fin.CP, q-fin.EC, q-fin.GN, q-fin.MF, q-fin.PM, q-fin.PR, q-fin.RM, q-fin.ST, q-fin.TR, stat.AP, stat.CO, stat.ME, stat.ML, stat.OT, stat.TH'

In [10]:
#count number of occurences of each category
value_counts = sorted([[category, categories_ls.count(category)] for category in categories_set], key = lambda x: x[1])

In [11]:
value_counts[-10:]

[['math.MP', 63403],
 ['math-ph', 63403],
 ['cond-mat.mtrl-sci', 67384],
 ['cs.LG', 68953],
 ['cond-mat.mes-hall', 70691],
 ['gr-qc', 84649],
 ['astro-ph', 105380],
 ['quant-ph', 107920],
 ['hep-th', 140204],
 ['hep-ph', 153126]]

In [2]:
data = data.drop([1636693, 1638090, 1637995])
data.to_pickle('abstract.pkl')

### Making Table with Authors and their paper

In [14]:
authors_array = np.array([data.id, data.authors_cleaned]).T

In [22]:
authors_dict = {}
for article in authors_array:
    ID = article[0]
    for author in article[1]:
        if author in authors_dict.keys():
            authors_dict[author].append(ID)
        else:
            authors_dict[author] = [ID]

In [40]:
authors_array = np.array([data.categories, data.authors_cleaned]).T
category_dict = {}
for article in authors_array:
    cat = article[0]
    cat = re.split('\s+', cat)
    for author in article[1]:
        if author in category_dict.keys():
            category_dict[author].extend(cat)
        else:
            category_dict[author] = cat

In [41]:
for key, value in category_dict.items():
    newvalue = list(set(value))
    category_dict[key] = newvalue

In [42]:
authors_data = pd.DataFrame()
authors_data['Authors'] = authors_dict.keys()
authors_data['Articles'] = authors_dict.values()
authors_data['Categories'] = category_dict.values()

In [47]:
authors_data['Num_Articles'] = [len(articles) for articles in authors_data.Articles]
authors_data['Num_Categories'] = [len(categories) for categories in authors_data.Categories]

In [49]:
authors_data.to_pickle('authors.pkl')
#data.to_pickle('abstract.pkl')

### Making Author Graph

In [ ]:
authors_array = np.array([data.id, data.authors_cleaned]).T
authors_dict = {}
for article in authors_array:
    ID = article[0]
    cycle = len(article[1])
    i = 0
    for author in article[1]:
        for i in range(cycle):
            if author == article[1][i]:
                continue
            elif article[1][i] in authors_dict.keys():
                addon = set(authors_dict[article[1][i]])
                addon.add(author)
                authors_dict[article[1][i]] = list(addon)
            else:
                authors_dict[article[1][i]] = [author]

In [ ]:
authors_graph = pd.DataFrame()
authors_graph['Authors'] = authors_dict.keys()
authors_graph['Collaborators'] = authors_dict.values()

In [7]:
data_ref = authors_graph.T.to_dict(orient='list') 

In [12]:
dict_nodes = {v[0]:v[1] for k, v in data_ref.items()}

In [14]:
len(dict_nodes)

1069645

In [15]:
import networkx as nx
from networkx.readwrite import json_graph

G = nx.from_dict_of_lists(dict_nodes)

In [16]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1069645
Number of edges: 42624678
Average degree:  79.6987


In [17]:
nx.write_gpickle(G,"AuthorGraph.gpickle")